<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [13]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

C:\Users\John\Anaconda3\envs\DS-U4-S1\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [15]:
STOPWORDS = set(STOPWORDS).union(set(['amazon', 'like']))

def tokenize(text):
    
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

df['tokens'] = df['reviews.text'].apply(tokenize)
df['tokens']

0        [order, item, bad, quality, missing, backup, s...
1                         [bulk, expensive, way, products]
2                                 [duracell, price, happy]
3                  [work, brand, batteries, better, price]
4                 [batteries, long, lasting, price, great]
5        [bought, lot, batteries, christmas, amazonbasi...
6        [ive, problame, batteries, ordered, past, plea...
7        [looking, cheap, non, rechargeable, batteries,...
8        [hold, high, power, juice, energizer, duracell...
9        [amazonbasics, aa, aaa, batteries, appear, goo...
10       [basics, batteries, equal, superior, brand, on...
11       [started, getting, basic, batteries, liked, re...
12       [use, fish, tank, light, night, works, great, ...
13       [got, em, comment, good, job, good, price, qui...
14                      [things, need, aa, battery, great]
15       [thankful, able, great, price, better, shippin...
16                      [know, buy, brand, long, duracel

In [20]:
id2word = corpora.Dictionary(df['tokens'])
len(id2word.keys())

9620

In [21]:
id2word.filter_extremes(no_below=5, no_above=0.95)
len(id2word.keys())

3580

In [22]:
corpus = [id2word.doc2bow(token) for token in df['tokens']]

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=100,
                   workers=8
                  )

In [23]:
lda.print_topics()

[(0,
  '0.098*"old" + 0.089*"year" + 0.044*"loves" + 0.035*"tablet" + 0.026*"bought" + 0.019*"daughter" + 0.017*"love" + 0.017*"yr" + 0.016*"games" + 0.015*"time"'),
 (1,
  '0.068*"books" + 0.058*"read" + 0.037*"reading" + 0.032*"kindle" + 0.027*"size" + 0.027*"light" + 0.026*"perfect" + 0.018*"book" + 0.015*"love" + 0.011*"small"'),
 (2,
  '0.123*"kids" + 0.055*"tablet" + 0.037*"great" + 0.027*"case" + 0.025*"love" + 0.019*"children" + 0.018*"apps" + 0.016*"kid" + 0.016*"parental" + 0.015*"child"'),
 (3,
  '0.068*"games" + 0.056*"gift" + 0.047*"love" + 0.034*"bought" + 0.031*"great" + 0.031*"play" + 0.031*"tablet" + 0.027*"loves" + 0.018*"christmas" + 0.017*"purchased"'),
 (4,
  '0.212*"use" + 0.206*"easy" + 0.050*"tablet" + 0.032*"set" + 0.025*"great" + 0.025*"love" + 0.020*"navigate" + 0.012*"apps" + 0.011*"loves" + 0.011*"purchased"'),
 (5,
  '0.040*"happy" + 0.040*"tablet" + 0.025*"bought" + 0.022*"charge" + 0.022*"purchase" + 0.019*"ipad" + 0.016*"little" + 0.015*"slow" + 0.014*"

In [25]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\John\Anaconda3\envs\DS-U4-S1\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\Users\John\Anaconda3\envs\DS-U4-S1\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.236706 -0.094239       1        1  14.920269
9     -0.149156 -0.235746       2        1   7.803669
6     -0.187080  0.272379       3        1   7.778697
13     0.017048 -0.098634       4        1   7.415217
10     0.031620  0.062992       5        1   7.031016
0      0.226505 -0.005016       6        1   6.582312
2      0.131638  0.109619       7        1   6.076600
3      0.220323  0.074346       8        1   6.048309
8      0.013517 -0.100413       9        1   5.983247
1      0.074235 -0.118547      10        1   5.793887
7      0.135511 -0.088978      11        1   5.363057
5     -0.026521 -0.118668      12        1   5.238388
11    -0.106901  0.218811      13        1   4.737476
14    -0.266329  0.023082      14        1   4.693811
4      0.122296  0.099012      15        1   4.534051, topic_info=     Category         Freq         Term        Total  loglift  logprob
17    Default  7640.000000    batteries  7640.000000  30.0000  30.0000
92    Default  4502.000000          use  4502.000000  29.0000  29.0000
20    Default  8871.000000        great  8871.000000  28.0000  28.0000
16    Default  4763.000000        price  4763.000000  27.0000  27.0000
753   Default  3005.000000         easy  3005.000000  26.0000  26.0000
32    Default  5482.000000         good  5482.000000  25.0000  25.0000
190   Default  2503.000000         kids  2503.000000  24.0000  24.0000
2     Default  2212.000000      battery  2212.000000  23.0000  23.0000
2074  Default  2998.000000       kindle  2998.000000  22.0000  22.0000
519   Default  2128.000000          old  2128.000000  21.0000  21.0000
103   Default  1778.000000         year  1778.000000  20.0000  20.0000
2431  Default  7055.000000       tablet  7055.000000  19.0000  19.0000
26    Default  3187.000000       bought  3187.000000  18.0000  18.0000
27    Default  2234.000000          buy  2234.000000  17.0000  17.0000
1855  Default  2253.000000        loves  2253.000000  16.0000  16.0000
2085  Default  1517.000000        books  1517.000000  15.0000  15.0000
164   Default  1091.000000    recommend  1091.000000  14.0000  14.0000
650   Default  1709.000000        games  1709.000000  13.0000  13.0000
163   Default  2024.000000      product  2024.000000  12.0000  12.0000
883   Default  1031.000000         gift  1031.000000  11.0000  11.0000
22    Default  2456.000000         long  2456.000000  10.0000  10.0000
784   Default  1210.000000         read  1210.000000   9.0000   9.0000
57    Default  1077.000000         life  1077.000000   8.0000   8.0000
88    Default  3517.000000         love  3517.000000   7.0000   7.0000
193   Default  1249.000000          far  1249.000000   6.0000   6.0000
220   Default  1422.000000         best  1422.000000   5.0000   5.0000
19    Default  1281.000000        brand  1281.000000   4.0000   4.0000
1546  Default  1882.000000         apps  1882.000000   3.0000   3.0000
944   Default  1205.000000      reading  1205.000000   2.0000   2.0000
407   Default  1188.000000         play  1188.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
3293  Topic15     8.915280      elderly    11.103829   2.8740  -7.3340
1128  Topic15     8.164112        round    10.051979   2.8855  -7.4220
223   Topic15    96.301781       simple   271.540863   2.0569  -4.9542
2706  Topic15    38.684437    functions    84.376198   2.3137  -5.8663
2958  Topic15    13.064873  grandmother    18.954853   2.7214  -6.9518
1745  Topic15    25.977468     toddlers    50.106995   2.4366  -6.2645
1639  Topic15    34.314468           nd    76.551987   2.2912  -5.9862
3210  Topic15    16.119930      grandma    25.894062   2.6196  -6.7417
2431  Topic15   680.662231       tablet  7055.844727   0.7550  -2.9987
432   Topic15    78.228149        clear   279.721313   1.8194  -5.1621
185   Topic15    55.612782   convenient   170.767715   1.

### Amazon Reviews

* Battery reviews seem to have positive reviews for their lasting more so than negative.

* Products are highly recommended with good prices

* Tablets are a very common item as they show up in majority of topics



## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling